In [1]:
#pip install --user --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [2]:
#pip install nest_asyncio

In [3]:
import socket
import sys
import requests
import json
import twint
from datetime import datetime, timedelta
from time import sleep


In [4]:
import nest_asyncio
nest_asyncio.apply()
# Tweets de las 2 últimas horas
current_date = datetime.now()
current_end_date = current_date - timedelta(minutes=240)
def twint_to_pandas(columns):
 return twint.output.panda.Tweets_df[columns]
# Configuración de twint
c= twint.Config()
c.Username = "el_pais"
c.Since = current_end_date.strftime("%Y-%m-%d %H:%M:%S")
c.Until = current_date.strftime("%Y-%m-%d %H:%M:%S")
c.Pandas = True
# Run
twint.run.Search(c)
df_pd = twint_to_pandas(["date", "username", 
 "tweet", "hashtags", 
"nlikes"])

1597930097134260225 2022-11-30 12:26:37 +0000 <el_pais> Estas serán las pensiones que se cobrarán en 2023  https://t.co/beDFmhhgmX
1597926629237571584 2022-11-30 12:12:51 +0000 <el_pais> La mejor profesora del mundo es española y maestra de Educación Infantil desde hace 20 años: "Los niños no fallan, son menores. Si algo no funciona no es por el niño. Si el niño suspende, ha suspendido el docente"  https://t.co/AjC1Jj4xDc
1597924963209842691 2022-11-30 12:06:13 +0000 <el_pais> 📺 TV en DIRECTO | La jornada clave del grupo de Argentina, los disturbios tras el Irán contra Estados Unidos y toda la actualidad de #Qatar2022, en #UniversoMundial30N  https://t.co/yWw8mM60De   https://t.co/Vzcqgd56xb
1597924666617696257 2022-11-30 12:05:03 +0000 <el_pais> 📺 TV en DIRECTO | Santiago Romero (Unidas Podemos): "Las imágenes de EL PAÍS muestra de forma clara que quedaron personas presuntamente heridas en suelo español, y se ve una persona que parece que ha fallecido"  https://t.co/0rTRG60ECl  https:

**Procesar los tweets**

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pandasToSparkDF").getOrCreate()
df_pd = spark.createDataFrame(df_pd)
lines = df_pd.select("tweet")
print(lines)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


DataFrame[tweet: string]


**Procesar los tweets**

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
import re
def preprocessing(lines):
     words = lines.na.replace("", None)
     words = lines.na.drop()
     words = lines.withColumn("tweet", F.regexp_replace("tweet", r'http\S+', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", '@\w+', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", '#', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", 'RT', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", ':', ''))
     full_tweet = words.toJSON()
     return full_tweet 


**Función para enviar los tweets al socket**

In [7]:
def send_tweets_to_spark(data, tcp_connection):
     for row in data.collect():
         line = row + "\n"
         try:
             tcp_connection.send(line.encode("utf-8"))
         except:
             e = sys.exc_info()[0]
             print("Error connection: %s" % e)

**Crear el socket donde se almacena la información**

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
data = preprocessing(lines)
send_tweets_to_spark(data, conn)

Waiting for TCP connection...
